# Alberta: Population Growth vs. Education Spending (2012–2025)

Examining whether Alberta's education spending grew **proportionally** to its rising population over 13 years.

---

**Data sources**
- **Population**: Statistics Canada, Table 17-10-0009-01 — quarterly population estimates for Alberta
- **Education spending**: Alberta Budget Fiscal Plans (2012, 2013, 2023, 2024, 2025) — K-12 and post-secondary operating expenses

**Methodology note**

Education spending figures are available for fiscal years **2012-13, 2013-14, 2023-24, 2024-25, and 2025-26** only. Figures for 2014-15 through 2022-23 are not included. All spending figures are **nominal** (not inflation-adjusted). Population is aligned to Q1 (January) of each fiscal year's starting calendar year — e.g., fiscal year 2012-13 is matched to Alberta's January 2012 population.

## Key Findings

| Metric | 2012-13 / Q1 2012 | 2025-26 / Q1 2025 | Nominal Growth |
|---|---|---|---|
| Alberta Population | 3,822,425 | 4,988,181 | **+30.5%** |
| K-12 Operating Expense | $6,179M | $9,883M | **+59.9%** |
| Post-Secondary Operating Expense | $2,856M | $6,635M | **+132.3%** |
| Total Education Expense | $9,035M | $16,518M | **+82.8%** |
| Education Spending per Capita | ~$2,363 | ~$3,312 | **+40.1%** |

### Interpretation

All categories of education spending grew substantially faster than Alberta's population in nominal terms:

- **Total education spending (+82.8%)** grew nearly **2.7×** faster than population (+30.5%)
- **K-12 spending (+59.9%)** outpaced population growth by roughly **2:1**
- **Post-secondary spending (+132.3%)** more than doubled, growing at **4.3×** the rate of population growth
- **Per-capita education spending rose ~40%** — each Albertan now has significantly more government education spending attributed to them than in 2012

**Important caveat:** All figures are nominal. Inflation erodes real purchasing power — a portion of the spending increase reflects price-level changes across the economy rather than real gains in educational capacity or resources per student. Adjusting for CPI inflation would provide a more complete picture.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from IPython.display import display

# ── Dark infographic theme (consistent with project style) ──────────────────
sns.set_theme(style='darkgrid', context='notebook', font_scale=1.15)
plt.rcParams.update({
    'figure.facecolor': '#0D1117',
    'axes.facecolor':   '#161B22',
    'axes.edgecolor':   '#30363D',
    'axes.labelcolor':  '#E6EDF3',
    'text.color':       '#E6EDF3',
    'xtick.color':      '#8B949E',
    'ytick.color':      '#8B949E',
    'grid.color':       '#21262D',
    'grid.alpha':       0.6,
    'font.family':      'sans-serif',
    'savefig.facecolor':'#0D1117',
})

POP_COLOR  = '#F778BA'   # pink         — population
K12_COLOR  = '#58A6FF'   # bright blue  — K-12
PS_COLOR   = '#56D364'   # bright green — post-secondary
TOT_COLOR  = '#FFB703'   # amber        — total education

print('Setup complete')

In [ ]:
# ── Population: Q1 (January) snapshots, one per calendar year 2012-2025 ─────
pop_raw = pd.read_csv('17100009.csv')
pop_raw['REF_DATE'] = pd.to_datetime(pop_raw['REF_DATE'])

ab_pop = pop_raw[
    (pop_raw['GEO'] == 'Alberta') &
    (pop_raw['REF_DATE'].dt.month == 1) &
    (pop_raw['REF_DATE'].dt.year.between(2012, 2025))
][['REF_DATE', 'VALUE']].copy()

ab_pop.columns = ['Date', 'Population']
ab_pop['Year'] = ab_pop['Date'].dt.year
ab_pop = ab_pop[['Year', 'Population']].reset_index(drop=True)

print('Alberta Q1 (January) population snapshots:')
display(ab_pop)

In [ ]:
# ── Education spending: one headline estimate per budget year ─────────────────
# Source: Alberta Budget Fiscal Plans. 'Year' = starting calendar year of the
# fiscal period, aligned to Q1 population snapshot.
spending = pd.DataFrame([
    {'Fiscal_Year': '2012-13', 'Year': 2012, 'K12_M': 6179,  'PostSec_M': 2856},
    {'Fiscal_Year': '2013-14', 'Year': 2013, 'K12_M': 6210,  'PostSec_M': 2682},
    {'Fiscal_Year': '2023-24', 'Year': 2023, 'K12_M': 8836,  'PostSec_M': 5604},
    {'Fiscal_Year': '2024-25', 'Year': 2024, 'K12_M': 9252,  'PostSec_M': 6305},
    {'Fiscal_Year': '2025-26', 'Year': 2025, 'K12_M': 9883,  'PostSec_M': 6635},
])
spending['Total_M'] = spending['K12_M'] + spending['PostSec_M']

# ── Merge with population on starting calendar year ──────────────────────────
df = spending.merge(ab_pop, on='Year', how='left')

# ── Per-capita spending ($) ──────────────────────────────────────────────────
df['K12_PerCapita']     = (df['K12_M']     * 1_000_000) / df['Population']
df['PostSec_PerCapita'] = (df['PostSec_M'] * 1_000_000) / df['Population']
df['Total_PerCapita']   = (df['Total_M']   * 1_000_000) / df['Population']

# ── Index to 2012-13 baseline (= 100) ───────────────────────────────────────
base = df.iloc[0]
df['Pop_Index']     = (df['Population']  / base['Population'])  * 100
df['K12_Index']     = (df['K12_M']       / base['K12_M'])       * 100
df['PostSec_Index'] = (df['PostSec_M']   / base['PostSec_M'])   * 100
df['Total_Index']   = (df['Total_M']     / base['Total_M'])     * 100

print('=== Integrated dataset ===')
display(
    df[['Fiscal_Year', 'Year', 'Population', 'K12_M', 'PostSec_M', 'Total_M',
        'K12_PerCapita', 'PostSec_PerCapita', 'Total_PerCapita']].style.format({
        'Population':         '{:,.0f}',
        'K12_M':              '${:,.0f}M',
        'PostSec_M':          '${:,.0f}M',
        'Total_M':            '${:,.0f}M',
        'K12_PerCapita':      '${:,.0f}',
        'PostSec_PerCapita':  '${:,.0f}',
        'Total_PerCapita':    '${:,.0f}',
    })
)

In [ ]:
# ── Summary table: baseline vs latest ───────────────────────────────────────
first, last = df.iloc[0], df.iloc[-1]

col_first = first['Fiscal_Year'] + ' / Q1 ' + str(int(first['Year']))
col_last  = last['Fiscal_Year']  + ' / Q1 ' + str(int(last['Year']))

summary = pd.DataFrame({
    'Metric': [
        'Alberta Population',
        'K-12 Spending (nominal)',
        'Post-Secondary Spending (nominal)',
        'Total Education Spending (nominal)',
        'Total Education Spending per Capita',
    ],
    col_first: [
        f"{int(first['Population']):,}",
        f"${int(first['K12_M']):,}M",
        f"${int(first['PostSec_M']):,}M",
        f"${int(first['Total_M']):,}M",
        f"${first['Total_PerCapita']:,.0f}/person",
    ],
    col_last: [
        f"{int(last['Population']):,}",
        f"${int(last['K12_M']):,}M",
        f"${int(last['PostSec_M']):,}M",
        f"${int(last['Total_M']):,}M",
        f"${last['Total_PerCapita']:,.0f}/person",
    ],
    'Nominal Growth': [
        f"+{(last['Population'] / first['Population'] - 1) * 100:.1f}%",
        f"+{(last['K12_M']      / first['K12_M']      - 1) * 100:.1f}%",
        f"+{(last['PostSec_M']  / first['PostSec_M']  - 1) * 100:.1f}%",
        f"+{(last['Total_M']    / first['Total_M']    - 1) * 100:.1f}%",
        f"+{(last['Total_PerCapita'] / first['Total_PerCapita'] - 1) * 100:.1f}%",
    ],
})

print('=== Growth Summary: 2012-13 baseline vs 2025-26 ===')
display(summary.style.hide(axis='index'))

In [ ]:
# ── CHART 1: Indexed Growth (2012-13 = 100) ──────────────────────────────────
# The headline chart: shows how each metric grew relative to the 2012 baseline.
# Lines are drawn within each data era; the gap region is shaded.

era1 = df[df['Year'].isin([2012, 2013])]
era2 = df[df['Year'].isin([2023, 2024, 2025])]

series = [
    ('Pop_Index',     POP_COLOR, 'o', 'Population'),
    ('K12_Index',     K12_COLOR, 's', 'K-12 Spending'),
    ('PostSec_Index', PS_COLOR,  '^', 'Post-Secondary Spending'),
    ('Total_Index',   TOT_COLOR, 'D', 'Total Education Spending'),
]

fig, ax = plt.subplots(figsize=(14, 7))

for col, color, marker, label in series:
    # Era 1: 2012-2013 (solid line)
    ax.plot(era1['Year'], era1[col], color=color, linewidth=3,
            marker=marker, markersize=11, zorder=5, label=label)
    # Era 2: 2023-2025 (same style, no duplicate label)
    ax.plot(era2['Year'], era2[col], color=color, linewidth=3,
            marker=marker, markersize=11, zorder=5)

# Baseline reference
ax.axhline(100, color='#8B949E', linewidth=1.2, linestyle='--',
           alpha=0.55, label='2012-13 Baseline (= 100)')

# Shade the data gap (2013 to 2023)
ax.axvspan(2013.4, 2022.6, alpha=0.07, color='#8B949E', zorder=1)
ax.text(2018, 109, 'Data gap\n2014 – 2022', ha='center', fontsize=11,
        color='#8B949E', style='italic')

# End-point value labels (rightmost year)
for col, color, _, _ in series:
    val = df.iloc[-1][col]
    ax.text(2025.25, val, f'{val:.0f}', color=color,
            fontsize=10, fontweight='bold', va='center')

ax.set_xlim(2011, 2027)
ax.set_xticks(df['Year'].tolist())
ax.set_xlabel('Calendar Year (Q1 population snapshot)', fontsize=13, labelpad=10)
ax.set_ylabel('Index  (2012-13 = 100)', fontsize=13, labelpad=10)
ax.set_title(
    'Alberta: Population Growth vs. Education Spending\n'
    'Indexed to 2012-13 = 100  |  Nominal figures',
    fontsize=18, fontweight='bold', pad=20, color='white'
)

leg = ax.legend(fontsize=12, loc='upper left', framealpha=0.7, edgecolor='#30363D')
leg.get_frame().set_facecolor('#161B22')
for t in leg.get_texts():
    t.set_color('#E6EDF3')

sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig('budget_data/integration_indexed_growth.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved -> budget_data/integration_indexed_growth.png')

In [ ]:
# ── CHART 2: Per-Capita Education Spending ───────────────────────────────────
# How many dollars does Alberta spend on education per resident?

labels = df['Fiscal_Year'].values
x = np.arange(len(labels))
w = 0.38

fig, ax = plt.subplots(figsize=(14, 7))

bars1 = ax.bar(x - w / 2, df['K12_PerCapita'],    w, label='K-12',
               color=K12_COLOR, edgecolor='#0D1117', linewidth=1.5)
bars2 = ax.bar(x + w / 2, df['PostSec_PerCapita'], w, label='Post-Secondary',
               color=PS_COLOR,  edgecolor='#0D1117', linewidth=1.5)

# Value labels on each bar
for bars, color in [(bars1, K12_COLOR), (bars2, PS_COLOR)]:
    for bar in bars:
        h = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, h + 18,
                f'${h:,.0f}', ha='center', va='bottom',
                fontsize=9, fontweight='bold', color=color)

# Gap annotation between fiscal years 2013-14 and 2023-24 (positions 1 and 2)
gap_y = df['K12_PerCapita'].max() * 0.62
ax.axvline(1.5, color='#8B949E', linewidth=1.2, linestyle=':', alpha=0.55)
ax.text(1.5, gap_y, '9-year\ndata gap', ha='center', va='center',
        fontsize=10, color='#8B949E', style='italic',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='#161B22',
                  edgecolor='#30363D', alpha=0.85))

ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=12, fontweight='bold')
ax.set_ylabel('Spending per Capita ($)', fontsize=13, labelpad=10)
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f'${v:,.0f}'))
ax.set_title(
    'Alberta: Education Spending per Capita\n'
    'K-12 vs Post-Secondary  |  Nominal figures',
    fontsize=18, fontweight='bold', pad=20, color='white'
)

leg = ax.legend(fontsize=13, loc='upper left', framealpha=0.7, edgecolor='#30363D')
leg.get_frame().set_facecolor('#161B22')
for t in leg.get_texts():
    t.set_color('#E6EDF3')

sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig('budget_data/integration_per_capita.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved -> budget_data/integration_per_capita.png')

In [ ]:
# ── CHART 3: Total Growth Rates — 2012-13 baseline → 2025-26 ─────────────────
# Lollipop chart comparing % growth of each metric.
# The population growth line acts as the reference benchmark.

first, last = df.iloc[0], df.iloc[-1]

metrics = [
    'Population',
    'K-12 Spending',
    'Post-Secondary\nSpending',
    'Total Education\nSpending',
]
pcts = [
    (last['Population']  / first['Population']  - 1) * 100,
    (last['K12_M']       / first['K12_M']        - 1) * 100,
    (last['PostSec_M']   / first['PostSec_M']    - 1) * 100,
    (last['Total_M']     / first['Total_M']      - 1) * 100,
]
colors = [POP_COLOR, K12_COLOR, PS_COLOR, TOT_COLOR]

fig, ax = plt.subplots(figsize=(12, 6))
y = np.arange(len(metrics))

for i in range(len(metrics)):
    ax.hlines(y[i], 0, pcts[i], color=colors[i], linewidth=4, alpha=0.85)
    ax.scatter(pcts[i], y[i], color=colors[i], s=280, zorder=5,
               edgecolor='#0D1117', linewidth=2)
    ax.text(pcts[i] + 3, y[i], f'+{pcts[i]:.1f}%',
            va='center', fontsize=14, fontweight='bold', color=colors[i])

# Population growth as a vertical reference line
pop_pct = pcts[0]
ax.axvline(pop_pct, color=POP_COLOR, linewidth=1.5, linestyle='--', alpha=0.45)
ax.text(pop_pct + 1, len(metrics) - 0.55,
        f'Population\ngrowth ({pop_pct:.1f}%)',
        color=POP_COLOR, fontsize=9, alpha=0.85)

ax.set_yticks(y)
ax.set_yticklabels(metrics, fontsize=13, fontweight='bold')
ax.set_xlabel('Growth from 2012-13 Baseline (%)', fontsize=13, labelpad=10)
ax.set_xlim(-5, max(pcts) * 1.3)
ax.set_title(
    'Education Spending vs. Population Growth\n'
    '2012-13 to 2025-26  |  Nominal figures',
    fontsize=18, fontweight='bold', pad=20, color='white'
)
ax.axvline(0, color='#30363D', linewidth=1)

sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig('budget_data/integration_growth_rates.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved -> budget_data/integration_growth_rates.png')

In [ ]:
# ── Export integrated dataset to CSV ─────────────────────────────────────────
export_cols = [
    'Fiscal_Year', 'Year', 'Population',
    'K12_M', 'PostSec_M', 'Total_M',
    'K12_PerCapita', 'PostSec_PerCapita', 'Total_PerCapita',
    'Pop_Index', 'K12_Index', 'PostSec_Index', 'Total_Index',
]
df[export_cols].to_csv('budget_data/population_vs_spending.csv', index=False)

print('Exported -> budget_data/population_vs_spending.csv')
print()
print('Charts saved to budget_data/:')
print('  integration_indexed_growth.png')
print('  integration_per_capita.png')
print('  integration_growth_rates.png')